In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
df = pd.read_csv('water_potability.csv')


In [ ]:
df.fillna(df.mean(), inplace=True)

# Normalize numeric features
scaler = MinMaxScaler()
df[['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']] = scaler.fit_transform(df[['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']])

In [ ]:
# Step 2: Feature Selection
# Step 3: Model Training
X = df.drop('Potability', axis=1)
y = df['Potability']


In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build a deep learning model
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)


In [ ]:
# Step 4: Model Evaluation
# Make predictions on the testing set
y_pred = (model.predict(X_test) > 0.5).astype(int)


# Evaluate the model's performance

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)


21/21 [==============================] - 0s 2ms/step


In [ ]:
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')
print('Confusion Matrix:')
print(confusion_mat)

Accuracy: 0.68
Precision: 0.62
Recall: 0.39
F1-score: 0.48
Confusion Matrix:
[[352  60]
 [148  96]]


In [ ]:
model.save("water_model.h5")

In [ ]:
import pandas as pd
import requests
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Step 1: Data Preparation
df = pd.read_csv('water_potability.csv')

# Handle missing values if any
df.dropna(inplace=True)

# Step 2: Statistical Analysis
# Calculate statistical measures for each parameter
statistical_measures = df.describe()

# Step 3: Anomaly Detection Algorithm
# Select features for anomaly detection
features = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

# Scale the features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[features])

def generate_alert():
    # Implement your alert/notification logic here
    pass

# Train the anomaly detection algorithm (Isolation Forest)
anomaly_detector = IsolationForest(contamination=0.05)  # Adjust the contamination parameter as needed
anomaly_detector.fit(df_scaled)

# Step 4: Real-Time Monitoring
# Continuously monitor incoming water quality data from API
api_url = 'https://your-api-url.com'  # Replace with your actual API URL

while True:
    # Fetch new data from API
    response = requests.get(api_url)
    new_data = response.json()

    # Create a DataFrame from the new data
    new_data_df = pd.DataFrame(new_data)

    # Apply scaling to the new data
    new_data_scaled = scaler.transform(new_data_df[features])

    # Predict the anomaly score for the new data
    anomaly_score = anomaly_detector.predict(new_data_scaled)

    # Step 5: Alert Generation
    if anomaly_score == -1:
        # Anomaly detected, generate alert/notification
        generate_alert()
